In [1]:
"""
Import Statements: From PyTorch RL Tutorial
"""
import gym
import math
import random
import numpy as np
from itertools import count
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import nle
from collections import namedtuple, deque

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from utils.nle_metrics import NetHackMetricsEnv

env = NetHackMetricsEnv('joefile')

In [3]:
"""
This model stores each individual experience replay.
"""
class Experience():
    def __init__(self, state, action, next_state, reward):
        self.state - state
        self.action = action
        self.next_state = next_state
        self.reward = reward

In [4]:
StateTransition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

In [5]:
"""
This model stores replays of memories as a whole. It also holds the full set, and manages removing old memories.
"""
class ReplayMemory():
    def __init__(self, max_memory):
        self.memory = []
        self.max_memory = max_memory
    
    def push(self, newExp):
        if len(self.memory) >= self.max_memory:
            self.memory.pop(0)
        self.memory.append(newExp)
    
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)
    
    def __len__(self):
        return len(self.memory)

In [16]:
"""
DQN implementation, with convolutional layers, max pooling implemented.
"""
class DQNetwork(nn.Module):
    def __init__(self):
        super(DQNetwork, self).__init__()
        self.joe = nn.Sequential(
            nn.Conv2d(4, 16, kernel_size=3, padding='same'),
            nn.BatchNorm2d(16),
            nn.ReLU(), # 21 * 79 * 16
            nn.MaxPool2d(3, padding=(0,1)), # 7 * 27 * 16
            nn.Conv2d(16, 64, kernel_size=3, padding='same'),# 7 * 27 * 64
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(3, padding=(1,0)), # 3 * 9 * 64
            nn.Flatten(),
            nn.Linear(3*9*64, 3*64),
            nn.ReLU(),
            nn.Linear(3*64, 45),
        )
        
    def forward(self, x):
        logits = self.joe(x.float())
        return logits

In [17]:
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10
steps_done = 0
episode_durations = []
n_actions = env.action_space_size
print(n_actions)
# All of the batches

45


In [18]:
policy_net = DQNetwork().to(device)
target_net = DQNetwork().to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()
optimizer = optim.RMSprop(policy_net.parameters())
memory = ReplayMemory(10000)

In [19]:
def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            return policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)

In [20]:
def optimize_model(iter_num):
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = StateTransition(*zip(*transitions))

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
    if iter_num % 100 == 0:
        print('Iteration number ' + str(iter_num) + ' loss: ' + str(loss.item()))
    
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [21]:
def concat_state(state):
    state = np.stack((state['glyphs'], state['chars'], state['colors'], state['specials']))
    state = torch.tensor(np.expand_dims(state, axis=0), device=device)
    return state

# EPISODES
num_episodes = 1
for i_episode in range(num_episodes):
    print('Episode Number: ' + str(i_episode))
    # Initialize the environment and state
    obs = env.start('joemama')
    state = concat_state(obs)
    for t in count():
        # Select and perform an action
        action = select_action(state)
        obs, reward, done, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        
        # Keep track of current state and previous states 
        last_state = state
        state = concat_state(obs)

        # Store the transition in memory
        memory.push(StateTransition(last_state, action, state, reward))

        # Perform one step of the optimization (on the policy network)
        optimize_model(t)
        if done:
            episode_durations.append(t + 1)
            break
    env.finish()
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())
print('Joe Done Complete')

Episode Number: 0
tensor([[44]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[7]], device='cuda:0')
tensor([[11]], device='cuda:0')
tensor([[42]], device='cuda:0')
tensor([[28]], device='cuda:0')
tensor([[35]], device='cuda:0')
tensor([[20]], device='cuda:0')
tensor([[32]], device='cuda:0')
tensor([[31]], device='cuda:0')
tensor([[20]], device='cuda:0')
tensor([[4]], device='cuda:0')
tensor([[26]], device='cuda:0')
tensor([[16]], device='cuda:0')
tensor([[13]], device='cuda:0')
tensor([[10]], device='cuda:0')
tensor([[12]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[30]], device='cuda:0')
tensor([[1]], device='cuda:0')
tensor([[32]], device='cuda:0')
tensor([[36]], device='cuda:0')
tensor([[40]], device='cuda:0')
tensor([[8]], device='cuda:0')
tensor([[3]], device='cuda:0')
tensor([[37]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[7]], device='cuda:0')
tensor([[35]], device='cuda:0')
tensor([[40]], device='cuda:0')
tensor([[36]], device='cuda:0

KeyboardInterrupt: 